### **Import Libraries and Modules**

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as stats
from pandas.tseries.holiday import USFederalHolidayCalendar
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from zipfile import ZipFile


 ### **Loading the test and training data**

In [2]:
# Load training data from a CSV file
train = pd.read_csv("train_data.csv")

# Load testing data from a CSV file
test = pd.read_csv('test_data.csv')

# Print the count of missing values in each column of the training data
print("Missing values in training data:")
print(train.isna().sum())

print()

# Print the count of missing values in each column of the testing data
print("Missing values in testing data:")
print(test.isna().sum())

Missing values in training data:
UserID                        0
QuestionTiming                0
TimeUtc                       0
CurrentGameMode           17013
CurrentTask               17013
CurrentSessionLength          0
LastTaskCompleted         62605
LevelProgressionAmount    17018
QuestionType                  0
ResponseValue                 0
dtype: int64

Missing values in testing data:
UserID                        0
QuestionTiming                0
TimeUtc                       0
CurrentGameMode           10333
CurrentTask               10333
CurrentSessionLength          0
LastTaskCompleted         36801
LevelProgressionAmount    10335
QuestionType                  0
dtype: int64



# **Preprocessing and EDA (Exploratory Data Analysis)**


### **Removing negative values in CurrentSessionLength**

In [3]:
# Remove rows where 'CurrentSessionLength' is less than zero
# This might be necessary if negative values are errors and not valid data points
train = train.drop(train[train['CurrentSessionLength'] < 0].index)

In [4]:
train.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,ResponseValue
0,p1,User Initiated,2022-08-18 22:55:27,NaN,NaN,2,NaN,NaN,Wellbeing,509.0
1,p1,System Initiated,2022-08-18 23:38:31,NaN,NaN,0,NaN,NaN,Wellbeing,653.0
2,p1,User Initiated,2022-08-18 23:39:24,Career,HOME_VAN,1,WASH_PWVan,1.000000,Wellbeing,705.0
3,p1,System Initiated,2022-08-18 23:45:01,Career,RESIDENTIALSMALL_BACKYARD,6,WASH_PWVan,0.168267,Wellbeing,817.0
4,p1,System Initiated,2022-08-18 23:51:22,Career,RESIDENTIALSMALL_BACKYARD,13,WASH_PWVan,0.429364,Wellbeing,810.0


### **Removing outliers using IQR filtering in CurrentSessionLength**

In [5]:
# Calculate the first quartile (Q1)
Q1 = train['CurrentSessionLength'].quantile(0.25)

# Calculate the third quartile (Q3)
Q3 = train['CurrentSessionLength'].quantile(0.75)

# The Interquartile Range (IQR) is the difference between the third and first quartiles
IQR = Q3 - Q1

# Define lower and upper bounds for filtering outliers
# Outliers are defined as points below Q1 - 1.5*IQR or above Q3 + 1.5*IQR
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the data to include only values within the lower and upper bounds
train = train[(train['CurrentSessionLength'] >= lower_bound) & (train['CurrentSessionLength'] <= upper_bound)]

In [6]:
train.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,ResponseValue
0,p1,User Initiated,2022-08-18 22:55:27,NaN,NaN,2,NaN,NaN,Wellbeing,509.0
1,p1,System Initiated,2022-08-18 23:38:31,NaN,NaN,0,NaN,NaN,Wellbeing,653.0
2,p1,User Initiated,2022-08-18 23:39:24,Career,HOME_VAN,1,WASH_PWVan,1.000000,Wellbeing,705.0
3,p1,System Initiated,2022-08-18 23:45:01,Career,RESIDENTIALSMALL_BACKYARD,6,WASH_PWVan,0.168267,Wellbeing,817.0
4,p1,System Initiated,2022-08-18 23:51:22,Career,RESIDENTIALSMALL_BACKYARD,13,WASH_PWVan,0.429364,Wellbeing,810.0


### **Median Imputation for LevelProgressionAmount**

In [7]:
# Calculate the median of the 'LevelProgressionAmount' in the training data
median_value = train['LevelProgressionAmount'].median()

# Fill missing values in the 'LevelProgressionAmount' column of the training data with the calculated median
train['LevelProgressionAmount'].fillna(median_value, inplace=True)

# Calculate the median of the 'LevelProgressionAmount' for the testing data
median_value = test['LevelProgressionAmount'].median()

# Fill missing values in the 'LevelProgressionAmount' column of the testing data with its median
test['LevelProgressionAmount'].fillna(median_value, inplace=True)

In [8]:
train.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,ResponseValue
0,p1,User Initiated,2022-08-18 22:55:27,NaN,NaN,2,NaN,0.543301,Wellbeing,509.0
1,p1,System Initiated,2022-08-18 23:38:31,NaN,NaN,0,NaN,0.543301,Wellbeing,653.0
2,p1,User Initiated,2022-08-18 23:39:24,Career,HOME_VAN,1,WASH_PWVan,1.000000,Wellbeing,705.0
3,p1,System Initiated,2022-08-18 23:45:01,Career,RESIDENTIALSMALL_BACKYARD,6,WASH_PWVan,0.168267,Wellbeing,817.0
4,p1,System Initiated,2022-08-18 23:51:22,Career,RESIDENTIALSMALL_BACKYARD,13,WASH_PWVan,0.429364,Wellbeing,810.0


In [9]:
test.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType
0,p1,System Initiated,2022-08-28 15:50:22,NaN,NaN,0,NaN,0.571778,Wellbeing
1,p1,User Initiated,2022-08-28 16:05:02,Career,RECREATIONGROUND_SKATEPARK,14,NaN,0.563458,Wellbeing
2,p1,User Initiated,2022-09-07 03:31:50,NaN,NaN,0,NaN,0.571778,Wellbeing
3,p1,System Initiated,2022-09-08 01:30:05,NaN,NaN,0,NaN,0.571778,Wellbeing
4,p1,System Initiated,2022-09-08 01:43:45,Career,RECREATIONGROUND_SKATEPARK,13,NaN,0.829827,Wellbeing


### **Feature-Engineering TimeUtc**

In [10]:

# Convert the 'TimeUtc' column to datetime format to enable datetime operations
train['TimeUtc'] = pd.to_datetime(train['TimeUtc'])

# Extracting various time components from the 'TimeUtc' column
train['Second'] = train['TimeUtc'].dt.second  # Extract seconds from 'TimeUtc'
train['Minute'] = train['TimeUtc'].dt.minute  # Extract minutes from 'TimeUtc'
train['Hour'] = train['TimeUtc'].dt.hour      # Extract hours from 'TimeUtc'
train['Day'] = train['TimeUtc'].dt.day        # Extract day of month from 'TimeUtc'
train['DayOfWeek'] = train['TimeUtc'].dt.dayofweek  # Extract day of the week (Monday=0, Sunday=6)

# Determine if the day is a weekend (Saturday or Sunday)
train['IsWeekend'] = train['DayOfWeek'] >= 5  # Boolean value, True if day is a weekend

train['Month'] = train['TimeUtc'].dt.month    # Extract month from 'TimeUtc'
train['Quarter'] = train['TimeUtc'].dt.quarter  # Extract quarter from 'TimeUtc'

# Calculate the elapsed time in seconds from the earliest timestamp in the data
train['ElapsedTime'] = (train['TimeUtc'] - train['TimeUtc'].min()).dt.total_seconds()

# Creating cyclic features for 'Hour' to capture the cyclical nature of time
train['Hour_sin'] = np.sin(2 * np.pi * train['Hour'] / 24)  # Sine transformation for hour
train['Hour_cos'] = np.cos(2 * np.pi * train['Hour'] / 24)  # Cosine transformation for hour

# Creating cyclic features for 'DayOfWeek' to capture the cyclical nature of days in a week
train['DayOfWeek_sin'] = np.sin(2 * np.pi * train['DayOfWeek'] / 7)  # Sine transformation for day of the week
train['DayOfWeek_cos'] = np.cos(2 * np.pi * train['DayOfWeek'] / 7)  # Cosine transformation for day of the week

In [11]:
train.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,ResponseValue,...,Day,DayOfWeek,IsWeekend,Month,Quarter,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos
0,p1,User Initiated,2022-08-18 22:55:27,NaN,NaN,2,NaN,0.543301,Wellbeing,509.0,...,18,3,False,8,3,35134.0,-0.500000,0.866025,0.433884,-0.900969
1,p1,System Initiated,2022-08-18 23:38:31,NaN,NaN,0,NaN,0.543301,Wellbeing,653.0,...,18,3,False,8,3,37718.0,-0.258819,0.965926,0.433884,-0.900969
2,p1,User Initiated,2022-08-18 23:39:24,Career,HOME_VAN,1,WASH_PWVan,1.000000,Wellbeing,705.0,...,18,3,False,8,3,37771.0,-0.258819,0.965926,0.433884,-0.900969
3,p1,System Initiated,2022-08-18 23:45:01,Career,RESIDENTIALSMALL_BACKYARD,6,WASH_PWVan,0.168267,Wellbeing,817.0,...,18,3,False,8,3,38108.0,-0.258819,0.965926,0.433884,-0.900969
4,p1,System Initiated,2022-08-18 23:51:22,Career,RESIDENTIALSMALL_BACKYARD,13,WASH_PWVan,0.429364,Wellbeing,810.0,...,18,3,False,8,3,38489.0,-0.258819,0.965926,0.433884,-0.900969


In [12]:
# Convert the 'TimeUtc' column to datetime format to enable datetime operations
test['TimeUtc'] = pd.to_datetime(test['TimeUtc'])

# Extracting various time components from the 'TimeUtc' column
test['Second'] = test['TimeUtc'].dt.second  # Extract seconds from 'TimeUtc'
test['Minute'] = test['TimeUtc'].dt.minute  # Extract minutes from 'TimeUtc'
test['Hour'] = test['TimeUtc'].dt.hour      # Extract hours from 'TimeUtc'
test['Day'] = test['TimeUtc'].dt.day        # Extract day of month from 'TimeUtc'
test['DayOfWeek'] = test['TimeUtc'].dt.dayofweek  # Extract day of the week (Monday=0, Sunday=6)

# Determine if the day is a weekend (Saturday or Sunday)
test['IsWeekend'] = test['DayOfWeek'] >= 5  # Boolean value, True if day is a weekend

test['Month'] = test['TimeUtc'].dt.month    # Extract month from 'TimeUtc'
test['Quarter'] = test['TimeUtc'].dt.quarter  # Extract quarter from 'TimeUtc'

# Calculate the elapsed time in seconds from the earliest timestamp in the data
test['ElapsedTime'] = (test['TimeUtc'] - test['TimeUtc'].min()).dt.total_seconds()

# Creating cyclic features for 'Hour' to capture the cyclical nature of time
test['Hour_sin'] = np.sin(2 * np.pi * test['Hour'] / 24)  # Sine transformation for hour
test['Hour_cos'] = np.cos(2 * np.pi * test['Hour'] / 24)  # Cosine transformation for hour

# Creating cyclic features for 'DayOfWeek' to capture the cyclical nature of days in a week
test['DayOfWeek_sin'] = np.sin(2 * np.pi * test['DayOfWeek'] / 7)  # Sine transformation for day of the week
test['DayOfWeek_cos'] = np.cos(2 * np.pi * test['DayOfWeek'] / 7)  # Cosine transformation for day of the week

In [13]:
test.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,Second,...,Day,DayOfWeek,IsWeekend,Month,Quarter,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos
0,p1,System Initiated,2022-08-28 15:50:22,NaN,NaN,0,NaN,0.571778,Wellbeing,22,...,28,6,True,8,3,873634.0,-0.707107,-0.707107,-0.781831,0.623490
1,p1,User Initiated,2022-08-28 16:05:02,Career,RECREATIONGROUND_SKATEPARK,14,NaN,0.563458,Wellbeing,2,...,28,6,True,8,3,874514.0,-0.866025,-0.500000,-0.781831,0.623490
2,p1,User Initiated,2022-09-07 03:31:50,NaN,NaN,0,NaN,0.571778,Wellbeing,50,...,7,2,False,9,3,1693322.0,0.707107,0.707107,0.974928,-0.222521
3,p1,System Initiated,2022-09-08 01:30:05,NaN,NaN,0,NaN,0.571778,Wellbeing,5,...,8,3,False,9,3,1772417.0,0.258819,0.965926,0.433884,-0.900969
4,p1,System Initiated,2022-09-08 01:43:45,Career,RECREATIONGROUND_SKATEPARK,13,NaN,0.829827,Wellbeing,45,...,8,3,False,9,3,1773237.0,0.258819,0.965926,0.433884,-0.900969


### **Frequency Encoding UserID**

In [14]:
# Frequency encoding for 'UserID' based on its occurrence in the training data
# Calculate the frequency of each user ID and normalize it to create a probability distribution
frequency_encoding = train['UserID'].value_counts(normalize=True).to_dict()

# Map the frequency values to the 'UserID' in the training dataset
train['UserID_encoded'] = train['UserID'].map(frequency_encoding)

# Apply the same mapping to the 'UserID' in the testing dataset
test['UserID_encoded'] = test['UserID'].map(frequency_encoding)

# Fill missing values in the testing data 'UserID_encoded' column with 0
# Missing values might occur if some UserIDs in the test data were not present in the training data
test['UserID_encoded'].fillna(0, inplace=True)

In [15]:
train.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,ResponseValue,...,DayOfWeek,IsWeekend,Month,Quarter,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos,UserID_encoded
0,p1,User Initiated,2022-08-18 22:55:27,NaN,NaN,2,NaN,0.543301,Wellbeing,509.0,...,3,False,8,3,35134.0,-0.500000,0.866025,0.433884,-0.900969,0.000177
1,p1,System Initiated,2022-08-18 23:38:31,NaN,NaN,0,NaN,0.543301,Wellbeing,653.0,...,3,False,8,3,37718.0,-0.258819,0.965926,0.433884,-0.900969,0.000177
2,p1,User Initiated,2022-08-18 23:39:24,Career,HOME_VAN,1,WASH_PWVan,1.000000,Wellbeing,705.0,...,3,False,8,3,37771.0,-0.258819,0.965926,0.433884,-0.900969,0.000177
3,p1,System Initiated,2022-08-18 23:45:01,Career,RESIDENTIALSMALL_BACKYARD,6,WASH_PWVan,0.168267,Wellbeing,817.0,...,3,False,8,3,38108.0,-0.258819,0.965926,0.433884,-0.900969,0.000177
4,p1,System Initiated,2022-08-18 23:51:22,Career,RESIDENTIALSMALL_BACKYARD,13,WASH_PWVan,0.429364,Wellbeing,810.0,...,3,False,8,3,38489.0,-0.258819,0.965926,0.433884,-0.900969,0.000177


In [16]:
test.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,Second,...,DayOfWeek,IsWeekend,Month,Quarter,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos,UserID_encoded
0,p1,System Initiated,2022-08-28 15:50:22,NaN,NaN,0,NaN,0.571778,Wellbeing,22,...,6,True,8,3,873634.0,-0.707107,-0.707107,-0.781831,0.623490,0.000177
1,p1,User Initiated,2022-08-28 16:05:02,Career,RECREATIONGROUND_SKATEPARK,14,NaN,0.563458,Wellbeing,2,...,6,True,8,3,874514.0,-0.866025,-0.500000,-0.781831,0.623490,0.000177
2,p1,User Initiated,2022-09-07 03:31:50,NaN,NaN,0,NaN,0.571778,Wellbeing,50,...,2,False,9,3,1693322.0,0.707107,0.707107,0.974928,-0.222521,0.000177
3,p1,System Initiated,2022-09-08 01:30:05,NaN,NaN,0,NaN,0.571778,Wellbeing,5,...,3,False,9,3,1772417.0,0.258819,0.965926,0.433884,-0.900969,0.000177
4,p1,System Initiated,2022-09-08 01:43:45,Career,RECREATIONGROUND_SKATEPARK,13,NaN,0.829827,Wellbeing,45,...,3,False,9,3,1773237.0,0.258819,0.965926,0.433884,-0.900969,0.000177


 ### **Target Encoding UserID**

In [17]:
# Prepare target  for the 'UserID' based on the 'ResponseValue'
encoder = TargetEncoder(smoothing=0.7)  # Initialize the TargetEncoder with smoothing to balance categorical average vs prior

# Apply the encoder to the 'UserID' column of the training data
# The encoder learns the mean target value for each category in 'UserID'
train['UserID_targetencoded'] = encoder.fit_transform(train['UserID'], train['ResponseValue'])

# Transform the 'UserID' column of the testing data using the learned encoding
# This ensures consistency in encoding between training and testing datasets
test['UserID_targetencoded'] = encoder.transform(test['UserID'])

# Calculate the global mean of the 'ResponseValue' from the training data
# This is used to fill in missing values for any 'UserID' that appears in the test set but not in the training set
global_mean = train['ResponseValue'].mean()

# Fill missing values in the 'UserID_targetencoded' column of the testing data with the global mean
# Missing values may occur if 'UserID' in the test data wasn't present in the training data
test['UserID_targetencoded'].fillna(global_mean, inplace=True)

In [18]:
train.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,ResponseValue,...,IsWeekend,Month,Quarter,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos,UserID_encoded,UserID_targetencoded
0,p1,User Initiated,2022-08-18 22:55:27,NaN,NaN,2,NaN,0.543301,Wellbeing,509.0,...,False,8,3,35134.0,-0.500000,0.866025,0.433884,-0.900969,0.000177,726.122568
1,p1,System Initiated,2022-08-18 23:38:31,NaN,NaN,0,NaN,0.543301,Wellbeing,653.0,...,False,8,3,37718.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568
2,p1,User Initiated,2022-08-18 23:39:24,Career,HOME_VAN,1,WASH_PWVan,1.000000,Wellbeing,705.0,...,False,8,3,37771.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568
3,p1,System Initiated,2022-08-18 23:45:01,Career,RESIDENTIALSMALL_BACKYARD,6,WASH_PWVan,0.168267,Wellbeing,817.0,...,False,8,3,38108.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568
4,p1,System Initiated,2022-08-18 23:51:22,Career,RESIDENTIALSMALL_BACKYARD,13,WASH_PWVan,0.429364,Wellbeing,810.0,...,False,8,3,38489.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568


In [19]:
test.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,Second,...,IsWeekend,Month,Quarter,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos,UserID_encoded,UserID_targetencoded
0,p1,System Initiated,2022-08-28 15:50:22,NaN,NaN,0,NaN,0.571778,Wellbeing,22,...,True,8,3,873634.0,-0.707107,-0.707107,-0.781831,0.623490,0.000177,726.122568
1,p1,User Initiated,2022-08-28 16:05:02,Career,RECREATIONGROUND_SKATEPARK,14,NaN,0.563458,Wellbeing,2,...,True,8,3,874514.0,-0.866025,-0.500000,-0.781831,0.623490,0.000177,726.122568
2,p1,User Initiated,2022-09-07 03:31:50,NaN,NaN,0,NaN,0.571778,Wellbeing,50,...,False,9,3,1693322.0,0.707107,0.707107,0.974928,-0.222521,0.000177,726.122568
3,p1,System Initiated,2022-09-08 01:30:05,NaN,NaN,0,NaN,0.571778,Wellbeing,5,...,False,9,3,1772417.0,0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568
4,p1,System Initiated,2022-09-08 01:43:45,Career,RECREATIONGROUND_SKATEPARK,13,NaN,0.829827,Wellbeing,45,...,False,9,3,1773237.0,0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568


 ### **Aggregated summary statistcs with UserID and ResponseValue**

In [20]:
# Calculate aggregated features: mean, minimum, and maximum of 'ResponseValue' for each 'UserID'
# This provides a statistical summary of the 'ResponseValue' which can be useful features for models
agg_features = train.groupby('UserID')['ResponseValue'].agg(['mean', 'min', 'max']).reset_index()

# Rename the columns for clarity
agg_features.columns = ['UserID', 'UserID_mean', 'UserID_min', 'UserID_max']

# Merge the aggregated features back into the training data
# This enriches the training data with historical summary statistics which may be relevant for prediction
train = train.merge(agg_features, on='UserID', how='left')

# Merge the same aggregated features into the testing data
# Ensures that the test data has the same feature enhancements as the training data
test = test.merge(agg_features, on='UserID', how='left')

# Fill missing values in the test data
# Missing values may occur if some 'UserID's in the test data do not have corresponding records in the training data
for col in ['UserID_mean', 'UserID_min', 'UserID_max']:
    test[col].fillna(train[col].mean(), inplace=True)  # Use the mean of the train data as a neutral imputation value


In [21]:
train.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,ResponseValue,...,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos,UserID_encoded,UserID_targetencoded,UserID_mean,UserID_min,UserID_max
0,p1,User Initiated,2022-08-18 22:55:27,NaN,NaN,2,NaN,0.543301,Wellbeing,509.0,...,35134.0,-0.500000,0.866025,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
1,p1,System Initiated,2022-08-18 23:38:31,NaN,NaN,0,NaN,0.543301,Wellbeing,653.0,...,37718.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
2,p1,User Initiated,2022-08-18 23:39:24,Career,HOME_VAN,1,WASH_PWVan,1.000000,Wellbeing,705.0,...,37771.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
3,p1,System Initiated,2022-08-18 23:45:01,Career,RESIDENTIALSMALL_BACKYARD,6,WASH_PWVan,0.168267,Wellbeing,817.0,...,38108.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
4,p1,System Initiated,2022-08-18 23:51:22,Career,RESIDENTIALSMALL_BACKYARD,13,WASH_PWVan,0.429364,Wellbeing,810.0,...,38489.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0


In [22]:
test.head()

,UserID,QuestionTiming,TimeUtc,CurrentGameMode,CurrentTask,CurrentSessionLength,LastTaskCompleted,LevelProgressionAmount,QuestionType,Second,...,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos,UserID_encoded,UserID_targetencoded,UserID_mean,UserID_min,UserID_max
0,p1,System Initiated,2022-08-28 15:50:22,NaN,NaN,0,NaN,0.571778,Wellbeing,22,...,873634.0,-0.707107,-0.707107,-0.781831,0.623490,0.000177,726.122568,724.318182,486.0,876.0
1,p1,User Initiated,2022-08-28 16:05:02,Career,RECREATIONGROUND_SKATEPARK,14,NaN,0.563458,Wellbeing,2,...,874514.0,-0.866025,-0.500000,-0.781831,0.623490,0.000177,726.122568,724.318182,486.0,876.0
2,p1,User Initiated,2022-09-07 03:31:50,NaN,NaN,0,NaN,0.571778,Wellbeing,50,...,1693322.0,0.707107,0.707107,0.974928,-0.222521,0.000177,726.122568,724.318182,486.0,876.0
3,p1,System Initiated,2022-09-08 01:30:05,NaN,NaN,0,NaN,0.571778,Wellbeing,5,...,1772417.0,0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
4,p1,System Initiated,2022-09-08 01:43:45,Career,RECREATIONGROUND_SKATEPARK,13,NaN,0.829827,Wellbeing,45,...,1773237.0,0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0


 ### **Dropping features with low importance**

In [23]:
# List of columns to be dropped from the dataset
columns_to_drop = [
    'TimeUtc', 'QuestionTiming', 'CurrentGameMode',
    'CurrentTask', 'LastTaskCompleted',
    'LastTaskCompleted_Aggregated', 'QuestionType', 'UserID',
]

# Removing specified columns from the training dataset
# This is done to clean up the data by eliminating irrelevant or redundant features
for col in columns_to_drop:
    if col in train.columns:
        train.drop(columns=col, inplace=True)

# Repeat the process for the testing dataset
# Ensuring that both datasets have the same structure for consistent model training and evaluation
for col in columns_to_drop:
    if col in test.columns:
        test.drop(columns=col, inplace=True)


In [24]:
train.head()

,CurrentSessionLength,LevelProgressionAmount,ResponseValue,Second,Minute,Hour,Day,DayOfWeek,IsWeekend,Month,...,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos,UserID_encoded,UserID_targetencoded,UserID_mean,UserID_min,UserID_max
0,2,0.543301,509.0,27,55,22,18,3,False,8,...,35134.0,-0.500000,0.866025,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
1,0,0.543301,653.0,31,38,23,18,3,False,8,...,37718.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
2,1,1.000000,705.0,24,39,23,18,3,False,8,...,37771.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
3,6,0.168267,817.0,1,45,23,18,3,False,8,...,38108.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
4,13,0.429364,810.0,22,51,23,18,3,False,8,...,38489.0,-0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0


In [25]:
test.head()

,CurrentSessionLength,LevelProgressionAmount,Second,Minute,Hour,Day,DayOfWeek,IsWeekend,Month,Quarter,ElapsedTime,Hour_sin,Hour_cos,DayOfWeek_sin,DayOfWeek_cos,UserID_encoded,UserID_targetencoded,UserID_mean,UserID_min,UserID_max
0,0,0.571778,22,50,15,28,6,True,8,3,873634.0,-0.707107,-0.707107,-0.781831,0.623490,0.000177,726.122568,724.318182,486.0,876.0
1,14,0.563458,2,5,16,28,6,True,8,3,874514.0,-0.866025,-0.500000,-0.781831,0.623490,0.000177,726.122568,724.318182,486.0,876.0
2,0,0.571778,50,31,3,7,2,False,9,3,1693322.0,0.707107,0.707107,0.974928,-0.222521,0.000177,726.122568,724.318182,486.0,876.0
3,0,0.571778,5,30,1,8,3,False,9,3,1772417.0,0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0
4,13,0.829827,45,43,1,8,3,False,9,3,1773237.0,0.258819,0.965926,0.433884,-0.900969,0.000177,726.122568,724.318182,486.0,876.0


 ### **Decision Tree Regressor Model Training and Validation**

In [26]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Set the sample size for training (80% train, 20% validation)
sample_size = 0.8

# Define features (X) and target variable (y)
X = train.drop(columns=['ResponseValue'])  # Features (exclude the target variable)
y = train['ResponseValue']  # Target variable

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=(1 - sample_size), random_state=42)

# Initialize the DecisionTreeRegressor with specified hyperparameters
dt = DecisionTreeRegressor(
    max_depth=20,              # Maximum depth of the tree
    min_samples_split=10,      # Minimum number of samples required to split an internal node
    min_samples_leaf=3,        # Minimum number of samples required to be at a leaf node
    random_state=42            # Seed for reproducibility
)

# Train the DecisionTreeRegressor on the training data
dt.fit(X_train, y_train)

# Make predictions on the validation set
val_predictions = dt.predict(X_val)

# Evaluate the model performance using Mean Absolute Error (MAE)
val_mae = mean_absolute_error(y_val, val_predictions)
print("Mean Absolute Error on Validation Set:", val_mae)


Mean Absolute Error on Validation Set: 98.84674736856985


 ### **Lasso Model Training and Validation**

In [29]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Set the sample size for training (80% training, 20% validation)
sample_size = 0.8

# Define features (X) and target variable (y)
X = train.drop(columns=['ResponseValue'])  # Features (exclude the target variable)
y = train['ResponseValue']  # Target variable

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=(1 - sample_size), random_state=42)

# Initialize the Lasso model with an alpha value (regularization strength)
lasso = Lasso(alpha=0.1, random_state=42)  # You can adjust alpha based on regularization needs

# Train the Lasso model using the training data
lasso.fit(X_train, y_train)

# Make predictions on the validation set
val_predictions = lasso.predict(X_val)

# Evaluate the model performance using Mean Absolute Error (MAE)
val_mae = mean_absolute_error(y_val, val_predictions)
print("Mean Absolute Error on Validation Set:", val_mae)


Mean Absolute Error on Validation Set: 91.74851524387616


 ### **XGBoost Regressor Training and Validation**

In [31]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Set the sample size for training (80% training, 20% validation)
sample_size = 0.8

# Define features (X) and target variable (y)
X = train.drop(columns=['ResponseValue'])  # Features (exclude the target variable)
y = train['ResponseValue']  # Target variable

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=(1 - sample_size), random_state=42)

# Initialize the XGBoost Regressor with specified hyperparameters
xgbr = xgb.XGBRegressor(
    n_estimators=300,          # Number of boosting rounds (trees)
    max_depth=6,               # Maximum depth of a tree
    learning_rate=0.1,         # Step size shrinkage used to prevent overfitting
    subsample=0.8,             # Subsample ratio of the training instance
    colsample_bytree=0.8,      # Subsample ratio of columns when constructing each tree
    random_state=42            # Seed for reproducibility
)

# Train the XGBoost Regressor using the training data
xgbr.fit(X_train, y_train)

# Make predictions on the validation set
val_predictions = xgbr.predict(X_val)

# Evaluate the model performance using Mean Absolute Error (MAE)
val_mae = mean_absolute_error(y_val, val_predictions)
print("Mean Absolute Error on Validation Set:", val_mae)


Mean Absolute Error on Validation Set: 87.2864163597108


 ### **Generating and Packaging Model Predictions**

# Making predictions using the RandomForest model on the test dataset
test_predictions = rf.predict(test)

# Save the predictions to a CSV file without headers or row indices
# This is often required in submission formats for machine learning competitions
pd.DataFrame(test_predictions).to_csv('predicted.csv', index=False, header=False)

# Loading the predictions to check the number of rows and ensure all data has been correctly predicted
pred = pd.read_csv('predicted.csv')
print("Number of predictions made:", pred.shape[0])

# Creating a zip file to package the predictions, commonly used for handling file submissions or storage efficiency
with ZipFile('predictions.zip', 'w') as zipf:
    zipf.write('predicted.csv', arcname='predicted.csv')
